In [5]:
import sys
from pathlib import Path

parent_folder = str(Path.cwd().parent)
print(f"Adding {parent_folder} to sys.path")
if parent_folder not in sys.path:
    sys.path.insert(0, parent_folder)

Adding c:\Users\bergsmann\code\turing-game\fourmind\experiment to sys.path


In [17]:
RAW_DATA_FILE_PATH = Path(".").absolute().parent / "daten_severin_20250901.json"
RAW_DATA_FILE_PATH

WindowsPath('c:/Users/bergsmann/code/turing-game/fourmind/experiment/daten_severin_20250901.json')

In [22]:
import json
from typing import Any

from models import GameData

with open(RAW_DATA_FILE_PATH, "r") as f:
    raw_data: dict[str, Any] = json.load(f)

data = [GameData(**item) for item in raw_data]  # type: ignore

In [32]:
from models import Message

errors: list[str] = []
player_messages: dict[str, list[str]] = {
    "AllTalker": [],
    "fourminds": [],
    "Human": [],
}
for current_game in data:
    messages: list[Message] = current_game.messages
    player_mapping: dict[str, str] = {
        current_game.bots[0].color: current_game.bots[0].name,
        current_game.bots[1].color: current_game.bots[1].name,
        list(current_game.player_info.keys())[0]: "Human",
    }
    for message in messages:
        try:
            player_name: str = player_mapping[message.color]
        except KeyError as e:
            errors.append(e.args[0])
            continue
        player_messages[player_name].append(message.message)

In [35]:
print(f"Found {len(errors)} errors.")
print(f"Collected {len(player_messages['AllTalker'])} messages from AllTalker.")
print(f"Collected {len(player_messages['fourminds'])} messages from fourminds.")
print(f"Collected {len(player_messages['Human'])} messages from Human.")

Found 156 errors.
Collected 1109 messages from AllTalker.
Collected 876 messages from fourminds.
Collected 738 messages from Human.


In [36]:
with open("player_messages.json", "w", encoding="utf-8") as outfile:
    json.dump(player_messages, outfile, ensure_ascii=False, indent=2)